In [1]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [4]:
#error = pd.read_pickle('scripts/cv_results/actions_e20/errors_search3.p')
#error2 = pd.read_pickle('scripts/cv_results/actions_e20/search3/')
#error3 = pd.read_pickle('scripts/cv_results/actions_e20/search2/res2.p')
param = pd.read_pickle('scripts/cv_results/actions_e20/errors_search3.p')
#error['mean'] = error.apply(np.mean, axis = 1)
# error = error.astype(np.float64)
# error['mean'] = error.mean(1)
# error['std'] = error.std(1)

In [14]:
param.loc[40:,param.columns.str.contains('valid')]

,valid-error-mean0,valid-error-std0,valid-error-mean1,valid-error-std1
40,0.141933,0.003387,0.140720,0.003697
41,0.141853,0.003269,0.140685,0.003790
42,0.141732,0.003379,0.140617,0.003625
43,0.141797,0.003300,0.140652,0.003713
44,0.141810,0.003406,0.140615,0.003655
45,0.141780,0.003481,0.140505,0.003550
46,0.141672,0.003471,0.140405,0.003698
47,0.141647,0.003412,0.140383,0.003752
48,0.141422,0.003479,0.140320,0.003725
49,0.141452,0.003497,0.140263,0.003718


In [35]:
s = pd.DataFrame()

In [3]:
param

,subsample,eta,colsample_bytree,max_depth
0,0.7,0.16,0.3,6
1,0.7,0.16,0.6,6


In [12]:
a = pd.concat([error2[10:]['mean'], error[10:]['mean']], axis = 1)
b = pd.concat([a, error3[10:]['mean']], axis = 1)

In [13]:
b

,mean,mean,mean
10,0.142613,0.142788,0.142330
11,0.142481,0.142621,0.142176
12,0.142326,0.142450,0.142154
13,0.142035,0.142180,0.142068
14,0.141909,0.142029,0.141897
15,0.141778,0.141921,0.141801
16,0.141733,0.141867,0.141792
17,0.141730,0.141863,0.141780
18,0.141594,0.141714,0.141910
19,0.141370,0.141533,0.142014


In [15]:
error3.to_pickle('scripts/cv_results/actions_e20/params1_err.p')

In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [142]:
#pd.isnull(sessions.groupby('user_id')['secs_elapsed'].sum()) == False
sessions.loc[sessions.user_id == '00allnceb8',:]
#sessions.loc[sessions.user_id == 'd1mm9tcy42',:]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1150481,00allnceb8,create,submit,signup,-unknown-,NaN


In [8]:
#find most popular actions
sessions.groupby('user_id').count()

NameError: name 'sessions' is not defined

In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)